In [1]:
import pandas as pd
import requests

In [2]:
dataset_name = 'gee'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [3]:
xlsx = requests.get('https://www.gov.br/mcti/pt-br/acompanhe-o-mcti/cgcl/clima/arquivos/arquivos_bi/5-aba/4cn_uf_agropecuaria.xlsx')
print(xlsx.raise_for_status())  # Verifica se o download foi bem-sucedido

# Salvar o arquivo .xlsx no disco
with open(f'{raiz}/bronze_mcti_{dataset_name}.xlsx', 'wb') as arquivo:
    arquivo.write(xlsx.content)

None


### Prata

Emissões de Gases do Efeito Estufa (GEE) do setor agropecuário

Unidade: Tonelada

In [4]:
df_prata = pd.DataFrame() # Criação de dataframe vazio para uso posterior

# Abas das tabelas na planilha bronze
tabelas = pd.ExcelFile(f'{raiz}/bronze_mcti_{dataset_name}.xlsx').sheet_names

In [5]:
for composto in tabelas:
    df = (pd.read_excel(f'{raiz}/bronze_mcti_{dataset_name}.xlsx',
                        sheet_name=composto,
                        header=5, 
                        nrows=27, 
                        index_col=0
                        )
                        .rename_axis('uf')
                        .reset_index()
                        .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet') # Banco de dados com geocódigos
                            .filter(['UF-id', 'UF-nome'])
                            .drop_duplicates(), # Exclusão de duplicatas (geradas por ser um BD com municípios)
                            how='left',
                            left_on='uf',
                            right_on='UF-nome'
                                )
                        .drop(columns=['uf', 'UF-nome']) # Exclusão de colunas desnecessárias para o prosseguimento
                        .melt(
                            id_vars='UF-id', # Coluna a ser mantida
                            var_name='data', # Nome da nova coluna com os dados transformados em linhas
                            value_name='valor' # Nome da nova coluna com os valores das antigas colunas
                                )
                        .query('valor != 0.0') # Exclusão de registros que possuam valor zerado
                        .rename(columns={'UF-id':'geocodigo'})
                        .assign(**{
                            'data': lambda df: pd.to_datetime(df['data'], format='%Y'), # Conversão de número inteiro para formato de data
                            'fonte': f'Emissão de {composto}', # Criação de nova coluna de informação do que representa o dado
                                })
                    )
    df_prata = (pd.concat([df_prata, df],
                         ignore_index=True)
                )

df_prata = (df_prata.convert_dtypes()
            .sort_values(by=['geocodigo', 'data']) # Organização dos dados
            .set_index(['geocodigo', 'data', 'fonte'])
            )

df_prata.to_parquet(f'{raiz}/prata_mcti_{dataset_name}.parquet')

### Ouro

In [3]:
df = pd.read_parquet(f'{raiz}/prata_mcti_{dataset_name}.parquet').reset_index()

(df.to_json(
    f'{raiz}/ouro_{dataset_name}.json',
    orient='records',
    date_format='iso',
    date_unit='s',
    indent=4,
    force_ascii=False
            )
 )